In [1]:
# Import system modules
import arcpy
from arcpy import env
arcpy.CheckOutExtension("Spatial")
arcpy.env.overwriteOutput = True

In [2]:
##folder
Res_fld = r"G:\UEP\GIS\Modules\GIS for Climate Change\LM5\MyProject13"

In [3]:
##workspace
env.workspace = r"G:\UEP\GIS\Modules\GIS for Climate Change\LM5\MyProject13\MyProject13.gdb"

In [4]:
## Set Geoprocessing environments
arcpy.env.snapRaster = "DEM_fill"
arcpy.env.extent = "DEM_fill"

In [5]:
print ("Preparing slope ...")
arcpy.Slope_3d("DEM_fill", "rSlope", "DEGREE")

Preparing slope ...


<Result 'G:\\UEP\\GIS\\Modules\\GIS for Climate Change\\LM5\\MyProject13\\MyProject13.gdb\\rSlope'>

In [6]:
print ("Reclassify - 10 degrees slopes = 1")
arcpy.gp.Reclassify_sa("rSlope", "Value", "0 10 NODATA;10 90 1", "rSlope10", "DATA")

Reclassify - 10 degrees slopes = 1


<geoprocessing server result object object at 0x00000199691774E0>

In [7]:
print ("Prepare the Flow Accumulation Raster ...")
print ("    Generate a MFD  raster")
FlowDirMFD = Res_fld + "FlowDirMFD.crf"
arcpy.gp.FlowDirection_sa("DEM_fill", FlowDirMFD, "NORMAL", "", "MFD")


Prepare the Flow Accumulation Raster ...
    Generate a MFD  raster


<geoprocessing server result object object at 0x000001996BF69660>

In [8]:
print ("    Generate Flow Accumulation Raster")
arcpy.gp.FlowAccumulation_sa(FlowDirMFD, "rFlowAccMFD", "", "INTEGER", "MFD")


    Generate Flow Accumulation Raster


<geoprocessing server result object object at 0x000001996C1742D0>

In [9]:
print ("    Reclassify the FlowAcc raster to highlight surface flow")
arcpy.gp.Reclassify_sa("rFlowAccMFD", "Value", "-2147483648 200 NODATA;200 2009479709 1", "rFlowAcc200MFD", "DATA")

    Reclassify the FlowAcc raster to highlight surface flow


<geoprocessing server result object object at 0x000001996C1744B0>

In [13]:
print ("Preparing sediment layer")
print ("    Clip sediment feature class to study area...")
arcpy.Clip_analysis("fcSediments", "MyWatershed_Land", "fcSediments_Clip")
print ("    Join field with erosion prone ranking")
arcpy.JoinField_management("fcSediments_Clip", "losmassetype", "LookUpTable4Erosion", "Kode", ["Kategori"])

Preparing sediment layer
    Clip sediment feature class to study area...
    Join field with erosion prone ranking


<Result 'fcSediments_Clip'>

In [14]:
## seperate the sediments most prone for erosion (kategori = 3) - change the expression to make selection for also lower values erodeness kategories 
arcpy.analysis.Select("fcSediments_Clip", "fcSediments_Cat3", "Kategori > 2")
arcpy.PolygonToRaster_conversion("fcSediments_Cat3", "Kategori", "rSediments_temp", "MAXIMUM_AREA", "", 1.0)
arcpy.gp.Reclassify_sa("rSediments_temp", "Value", "0 NODATA;3 1", "rSediments_Cat3", "DATA")

<geoprocessing server result object object at 0x000001996C238F90>

In [15]:
print ("Preparing erosion layer")
print ("    Identify erosion prone areas")
arcpy.gp.Times_sa("rSediments_Cat3", "rFlowAcc200MFD", "rTemp1")
arcpy.gp.Times_sa("rTemp1", "rSlope10", "rTemp2")
arcpy.gp.Reclassify_sa("rTemp2", "Value", "0 NODATA;1 1", "rErosjon_Cat3", "DATA")
print ("    Convert erosion exposed areas to polygon feature")
arcpy.RasterToPolygon_conversion("rErosjon_Cat3", "fcErosjon_tmp", "NO_SIMPLIFY", "Value", "SINGLE_OUTER_PART", "")
print ("    Remove all erosion-polygon that only occupy one pixel's area")
arcpy.analysis.Select("fcErosjon_tmp", "fcErosjon_Cat3", "Shape_Area > 1")


Preparing erosion layer
    Identify erosion prone areas
    Convert erosion exposed areas to polygon feature
    Remove all erosion-polygon that only occupy one pixel's area


<Result 'G:\\UEP\\GIS\\Modules\\GIS for Climate Change\\LM5\\MyProject13\\MyProject13.gdb\\fcErosjon_Cat3'>